# Spike: Multiprocessing Analysis

This notebook tests parallelizing our per-position analysis over multiple CPU cores, using a `Pool` of Stockfish engine processes.


In [28]:
import multiprocessing as mp
import chess, chess.engine
from dotenv import load_dotenv
import os
import csv
from typing import Dict, Tuple

In [23]:
load_dotenv()

True

In [24]:
STOCKFISH_PATH = os.environ.get("STOCKFISH_PATH")

In [25]:
with open("data/opening_positions.csv", "r") as f:
    reader = csv.reader(f)
    opening_positions: Dict[str, Tuple[str, str]] = {
        row[0]: (row[1], row[2]) for row in reader
    }
    opening_positions.pop("FEN", None)  # Remove header if present
    opening_positions.pop("BestMove", None)  # Remove header if present
    opening_positions.pop("Score", None)  # Remove header if present

In [26]:
opening_positions

{'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2': ('g1f3',
  '0.34'),
 'rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2': ('g1f3',
  '0.31'),
 'rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2': ('d2d4',
  '0.39'),
 'rnbqkbnr/pp1ppppp/2p5/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2': ('d2d4',
  '0.45'),
 'rnbqkbnr/pppppp1p/6p1/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2': ('d2d4',
  '0.63'),
 'rnbqkbnr/ppp1pppp/8/3p4/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2': ('e4d5',
  '0.62'),
 'rnbqkb1r/pppppppp/5n2/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 1 2': ('e4e5',
  '0.75'),
 'rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2': ('c2c4',
  '0.25'),
 'rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2': ('c2c4',
  '0.23'),
 'rnbqkbnr/ppp2ppp/4p3/3p4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 3': ('b1c3',
  '0.27'),
 'rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 3': ('g1f3',
  '0.3'),
 'rnbqkb1r/pppppp1p/5np1/8/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 3'

In [19]:
def worker(fen):
    engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)
    info = engine.analyse(chess.Board(fen), chess.engine.Limit(depth=20))
    engine.quit()
    return fen, info["score"].white().score()

if __name__ == "__main__":
    pool = mp.Pool(processes=4)
    results = pool.map(worker, opening_positions.keys())
    pool.close()
    pool.join()
    
    for fen, score in results:
        print(f"FEN: {fen}, Score: {score}")

Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/richardstrange/.pyenv/versions/3.12.10/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/richardstrange/.pyenv/versions/3.12.10/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/richardstrange/.pyenv/versions/3.12.10/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Users/richardstrange/.pyenv/versions/3.12.10/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'worker' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
 

KeyboardInterrupt: 

We have pickling issues!

To avoid the pickling issues in notebooks, we’ll use a `ThreadPoolExecutor` (each thread launches its own Stockfish process) so we can get concurrency without needing a separate script.


In [20]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [30]:
results = []
with ThreadPoolExecutor(max_workers=4) as pool:
    futures = [pool.submit(worker, fen) for fen in opening_positions.keys()]
    for fut in as_completed(futures):
        fen, score = fut.result()
        results.append((fen, score))

# Display results
for fen, score in results:
    print(f"{fen[:30]}… → {score:+d} cp")

rnbqkbnr/pppp1ppp/4p3/8/4P3/8/… → +41 cp
rnbqkbnr/pp1ppppp/8/2p5/4P3/8/… → +38 cp
rnbqkbnr/pppp1ppp/8/4p3/4P3/8/… → +31 cp
rnbqkbnr/pp1ppppp/2p5/8/4P3/8/… → +36 cp
rnbqkbnr/ppp1pppp/8/3p4/4P3/8/… → +65 cp
rnbqkbnr/pppppp1p/6p1/8/4P3/8/… → +66 cp
rnbqkbnr/ppp1pppp/8/3p4/3P4/8/… → +22 cp
rnbqkb1r/pppppppp/5n2/8/4P3/8/… → +75 cp
rnbqkb1r/pppppppp/5n2/8/3P4/8/… → +22 cp
rnbqkbnr/ppp2ppp/4p3/3p4/2PP4/… → +27 cp
rnbqkbnr/pp2pppp/2p5/3p4/2PP4/… → +33 cp
rnbqkb1r/pppppp1p/5np1/8/2PP4/… → +35 cp
rnbqkb1r/pppp1ppp/4pn2/8/2PP4/… → +22 cp
rnbqkbnr/ppppp1pp/8/5p2/3P4/8/… → +55 cp
rnbqkbnr/pp1ppppp/8/2p5/3P4/8/… → +78 cp
rnbqkbnr/pppp1ppp/8/4p3/2P5/8/… → +20 cp
rnbqkbnr/pppppp1p/6p1/8/3P4/8/… → +60 cp
rnbqkbnr/pppp1ppp/4p3/8/2P5/8/… → +26 cp
rnbqkb1r/pppppppp/5n2/8/2P5/8/… → +28 cp
rnbqkbnr/pp1ppppp/8/2p5/2P5/8/… → +30 cp
rnbqkbnr/ppp1pppp/8/3p4/8/5N2/… → +24 cp
rnbqkb1r/pppppppp/5n2/8/8/5N2/… → +25 cp
rnbqkbnr/pppppp1p/6p1/8/2P5/8/… → +27 cp
rnbqkbnr/pp1ppppp/8/2p5/8/5N2/… → +35 cp
rnbqkbnr/pppp1pp